In [16]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
from torchmetrics.functional import r2_score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, EnsemblePGLSModel
import timm

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [18]:
torch.cuda.empty_cache()

In [19]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [20]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224
    transforms.ToTensor(),         # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

batch_size = 16     # use 4 if problem with GPU memory

In [21]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [22]:
original_stds = torch.from_numpy(original_stds.values).float()
original_means = torch.from_numpy(original_means.values).float()
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [23]:
effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
# effnet = timm.create_model(
#     'efficientnet_b3.ra2_in1k',
#     pretrained=True,
#     num_classes=0,
# )

model = PGLSModel(effnet, tabular_input_size)
metric = R2Score()
criterion = torch.nn.MSELoss()

torch.cuda.empty_cache()

/home/szaryvip/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(),maximize=False, lr=1e-4, weight_decay=1e-5)

model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())

Epoch 0
1.1051925420761108
tensor(-0.9071)
1.2168571949005127
tensor(-1.7332)
0.8358830213546753
tensor(-1.1172)
0.8096823692321777
tensor(-0.2517)
0.49530643224716187
tensor(-0.3218)
1.4977473020553589
tensor(-0.0994)
0.5923575758934021
tensor(-0.0847)
0.8146198987960815
tensor(-0.0890)
0.5992886424064636
tensor(-0.1003)
2.263920307159424
tensor(-0.0256)
1.2098064422607422
tensor(-0.0268)
1.5381050109863281
tensor(-0.0175)
0.7608194351196289
tensor(-0.0168)
1.028751254081726
tensor(-0.0162)
1.1056541204452515
tensor(-0.0137)
0.761627197265625
tensor(-0.0143)
1.0176399946212769
tensor(-0.0115)
1.0763237476348877
tensor(-0.0108)
0.8506696224212646
tensor(-0.0092)
1.0355902910232544
tensor(-0.0071)
1.0680832862854004
tensor(-0.0083)
0.968683123588562
tensor(-0.0076)
0.7811449766159058
tensor(-0.0067)
0.8477801084518433
tensor(-0.0061)
0.8384444117546082
tensor(-0.0049)
0.9222189784049988
tensor(-0.0035)
1.1133679151535034
tensor(-0.0043)
1.4771875143051147
tensor(-0.0044)
0.9377169609069

KeyboardInterrupt: 

In [ ]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
with torch.no_grad():
    for data in val_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        accumulated_loss += loss.item()
        iterations += 1
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

Average loss on validation set:  0.8942889505368801
Average R2 on validation set:  0.08000875921474278


In [ ]:
predictions = []
model.eval()
model.to(device)
with torch.no_grad():
    for data in test_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        outputs = outputs.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [ ]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            pred = [p.item() for p in pred]
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [ ]:
save_predictions(predictions, test_tabular_data, "predictions.csv")

In [31]:
# effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
effnet = timm.create_model(
    'efficientnet_b0.ra_in1k',
    pretrained=True,
    num_classes=0,
)
xception = timm.create_model('inception_v4.tf_in1k', pretrained=True, num_classes=0)
densenet = timm.create_model('densenet121.ra_in1k', pretrained=True, num_classes=0)

In [32]:

metric = R2Score()
criterion = torch.nn.MSELoss()
# criterion = r2_score
ensemble_model = EnsemblePGLSModel([effnet, xception, densenet], tabular_input_size)
torch.cuda.empty_cache()

In [33]:
ensemble_model.to(device)

optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=1e-4, weight_decay=1e-5, maximize=False)

ensemble_model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())

Epoch 0
0.7720486521720886
tensor(0.0864)
0.9369663596153259
tensor(0.1629)
0.6722514629364014
tensor(0.1405)
0.5790714025497437
tensor(0.1288)
0.5463317632675171
tensor(0.1171)
0.8792229890823364
tensor(0.1082)
1.9145596027374268
tensor(0.1536)
1.043658971786499
tensor(0.1761)
0.9838432669639587
tensor(0.1706)
0.5145126581192017
tensor(0.1761)
0.42214035987854004
tensor(0.1742)
1.1196831464767456
tensor(0.1689)
0.6387522220611572
tensor(0.1837)
1.1976121664047241
tensor(0.1786)
1.774614691734314
tensor(0.1765)
1.6602424383163452
tensor(0.1787)
0.4541091322898865
tensor(0.1762)
0.5972698330879211
tensor(0.1739)
0.6405563354492188
tensor(0.1740)
0.4567156434059143
tensor(0.1774)
0.8463954925537109
tensor(0.1766)
1.0429967641830444
tensor(0.1793)
0.8064982295036316
tensor(0.1774)
0.527105450630188
tensor(0.1816)
0.4186338186264038
tensor(0.1854)
0.4238893985748291
tensor(0.1887)
0.8064765930175781
tensor(0.1878)
0.3417207896709442
tensor(0.1905)
0.6244075298309326
tensor(0.1898)
0.689086

KeyboardInterrupt: 